# How to read MDF files in Python

In this notebook, an example is given on how to read an MDF file using the Python *mdf_reader* package. Firstly, this is done by just reading all the data at once. Next, it is demonstrated how to make a choice of channels to read such that the data reading time can be reduced significantly. 

But first, let's start with importing some modules and setting up the information logger. 

In [ ]:
import os
import funcy
import matplotlib.pyplot as plt
import seaborn as sns

import mdf_reader.mdf_parser as mdf

sns.set("notebook")

file_name = "data/AMS_BALDER_110225T233000_UTC222959.mdf"
if not os.path.exists(file_name):
    # Dependent of from where we are running the notebook (inside the example directory or in the root),
    # we need to add one directory level up
    file_name = os.path.join("..", file_name)

We are ready to read the mdf date by just creating a *MDFParser* object with the *file_name* as input. We put the *print_duration()* around the reader only to show how long the reader needs to import all the data, but this is not required

In [ ]:
print("Reading the mdf file {}".format(file_name))
# Add the Timer only to show how long the reader takes
with funcy.print_durations("MDFParser"):
    mdf_obj = mdf.MDFParser(mdf_file=file_name)
print("Done")

mdf_obj.data.info()


The reading of an MDF file takes quite some time due to the binary conversion which needs to take place under the hood. In this case the reading time was 3.52 s. After we are done reading, all the data is stored in a pandas *DataFrame* which can be referred to as `mdf_obj.data`. We currently have loaded all 116 columns. 

We can have a look at the contents of the columns by using the *make_report()* method. This will show a table to screen with five columns:
1. A counter referring to the index of the table
2. An index referring to the position of the channel in the DataFrame
3. A flag *Loaded* to show if this channel has been loaded (now all set to true)
4. The (unique) record name is also used to refer to a channel in the DataFrame.
5. The label of the record giving a small description of the channel.


In [ ]:
names = mdf_obj.make_report()

As already mentioned, you can refer the dataframe as mdf_obj.data, and since this is a pandas DataFrame, we can use the pandas method *describe()* to give some statistical information on a channel. For instance, the MDF_Roll channel has the following statistics


In [ ]:
mdf_obj.data["MRU_Roll"].describe()

Use the *values* attribute to get the raw numpy array data. The first five values of the roll are for instance 


In [ ]:
mdf_obj.data["MRU_Roll"].values[:5]

These values could also be shown in the *pandas* way using the *head* method. This will plot the column data vs the index, which by default is the Date/Time. Here we demonstrate how to show the head of multiple columns. Note that you pass the column selection in a list, hence the double []


In [ ]:

mdf_obj.data[["MRU_Roll", "MRU_Pitch", "MRU_Heave"]].head(5)




## Speeding up the reading time

Quite ofter we are not interested in loading all the columns, but we just need a selection. The *mdf_parser* makes this easier by allowing to split up the reading of the MDF file: first the MDF header can be imported quickly, and then we make a choice of channels and import only the requirement. This will reduce the reading time significantly. 

#### Selecting the columns based on a search string

Start with reading the header of the file


In [ ]:
# first read the header dat only to get the column names
print("Reading header of {}".format(file_name))
with funcy.print_durations(f"MDFParser reading {file_name}"):
    mdf_object = mdf.MDFParser(file_name, verbose=1, import_data=False)

print("Done reading the header")



Since we had the import_data flag on False, the reading took only a few milliseconds. We can show the available data columns again by using the *make_report()* method just as we did above. The only difference is that the Loaded flag will be set to false

Now we know what columns are available we can make a selection by using the `set_column_selection` method. Suppose we want to plot the Roll along the journey

In [ ]:

mdf_object.set_column_selection(filter_list=["MRU_Roll"], include_date_time=True)

# now do the actual import
print("Importing the data from {}".format(file_name))
with funcy.print_durations("MDFParser"):
    #    mdf_object.import_data()
    mdf_object.import_data()

print("Done")
mdf_obj.data.info()
names2 = mdf_object.make_report(show_loaded_data_only=True)


The *include_date_time* flag was added to automatically include the DateTime field if available. The DateTime will be assigned to the index of the DataFrame, so we can now plot the Roll vs date time

In [ ]:

# plot one of the Roll vs. index
mdf_object.data.plot(y=["MRU_Roll"])
plt.ylabel('Roll [deg]')
plt.show()


The reading time has dropped to less than a second. The DateTime is implicitly imported (as we did not mention it in the filter list), so we can plot the Roll vs. the date

#### Selecting the columns based on the column names and labels

In the make_report() output it can be seen that each channel in the MDF file has a unique name which is not very self-explanatory (such as *BALDER_UI1_A130_VI18*) and a label with a description such as *HZ3 - AX - 03* which is the HZ3 location with the acceleration in x direction. We can also filter on the label since the filter_list argument consists of a list of regular expressions. The first regular expression is always applied on the channel *name*, whereas all the next regular expressions are applied on the channel *labels*. So we can select all the HZ3 accelerations by doing


In [ ]:
mdf_object.set_column_selection(filter_list=["^BALDER", "HZ3.*A[XYZ]"])
mdf_object.import_data()
names3 = mdf_object.make_report(show_loaded_data_only=True)


So in this case we have first selected all the channels with a record name starting with `^BALDER` (the `^` symbol matches the start of a string) and from that list we have selected all the channels with a label with `HZ3.*A[XYZ]`. Note that the `.*` matches a string of arbitrary length (also empty) with any character and the part with `[XYZ]` matches the `X` of `Y` or `Z` character. So using this filter list, we have now loaded the HZ3 AX and AY component (as AZ is not available). Note that the MRU_Roll component is still available from the previous round. 

We have used the *make_report()* method again to show the contents of the mdf_object data. The *show_loaded_data_only* flags is used to suppress all the columns which are available in the MDF file but have not been loaded yet. The column names in the table shown by *make_report* are returned to the *names* list. We can use this list to access the data frame column names of the column we want to use. In this case, we first extract the names of the HZ3 AX and AY column, which are stored in the one and two index of this names list (as you can see from the *cnt* column in the table). Get the names and plot the data

In [ ]:
hz3x_name = names3[1]
hz3y_name = names3[2]
ax = mdf_object.data.plot(y=[hz3x_name, hz3y_name])

# this is only required to replace the default labels (which are the column names) to
# something meaning full
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines[:2], ["HZ3 - AX", "HZ3 - AY"], loc="best")
plt.show()

We can extend the *filter_list* with more labels search patterns if we want. For instance, if you want to load all the channels from the RX4 and the Tower you can do

In [ ]:
mdf_object.set_column_selection(filter_list=["^BALDER", "Tower.*", "RX4"])
mdf_object.import_data()
mdf_object.make_report(show_loaded_data_only=True)

You can see that both the SB and PS Tower channels are loaded and also the RX4. The HZ3 was loaded in the previous call already. In this way, you can extend the channels being loaded.  

Remember that the first pattern in the *filter_list* (`^BALDER`) applies to the name of the record, all the following patterns apply to the labels. In case you want to select multiple columns based on the name field of the record you have to use the regular expression which `|`, which means `or`. For instance, load the latitude and longitude	

In [ ]:
mdf_object.set_column_selection(filter_list=["GPS_Lat.*|GPS_Lon.*"])
mdf_object.import_data()
mdf_object.make_report(show_loaded_data_only=True)

Now we have included the latitude and longitude column. We can display some information on the locations as follows

In [ ]:
mdf_object.data[["GPS_LatitudeDez", "GPS_LongitudeDez"]].describe()